In [ ]:
'''
Advent of Code 2021 - Day 4 - Giant Squid
==========================================

Part 1 - Bingo
==============

Goals:
1. Find the winning grid, based on the provided number stream
2. Find the last number called that gave the winning grid
3. Find the sum of all the unmarked numbers on the winning grid
4. Calculate the winning grid score (2. * 3.)
''' 

In [78]:
import numpy as np
import pandas as pd

In [84]:
# Number stream is the first line in the provided data

with open('input.txt', 'r') as file:
    number_stream = file.readline()
    number_stream = number_stream.strip().split(',')

In [85]:
with open('input.txt', 'r') as file:
    full = file.readlines()
    num_lines = len(full)

In [81]:
# Grids start at line 3 in the input data, each is 5 lines long with
# an empty line separating grids
# Each grid can be represented as a list of lists
# columns can be accessed by transposing the list of lists

line_start = 2
lines_to_read = 5
grids = []

while line_start < num_lines:
    with open('input.txt', 'r') as file:
        block = file.readlines()[line_start:line_start + lines_to_read]
        grid = []
        for line in block:
            grid.append(line.strip().split())
        line_start += 6
        grids.append(grid)

display(grids[0])  # verify first grid
display(grids[-1]) # verify last grid
print(len(grids))

trans = [list(i) for i in zip(*grids[0])] # verify column access 
display(trans)

[['62', '5', '77', '94', '75'],
 ['59', '10', '23', '44', '29'],
 ['93', '91', '63', '51', '74'],
 ['22', '14', '15', '2', '55'],
 ['78', '18', '95', '58', '57']]

[['1', '93', '61', '43', '39'],
 ['20', '67', '4', '58', '32'],
 ['99', '31', '72', '40', '6'],
 ['88', '19', '42', '52', '49'],
 ['35', '45', '65', '50', '91']]

100


[['62', '59', '93', '22', '78'],
 ['5', '10', '91', '14', '18'],
 ['77', '23', '63', '15', '95'],
 ['94', '44', '51', '2', '58'],
 ['75', '29', '74', '55', '57']]

In [82]:
end_slice = 5 
match = []  # when a match is found, grid idx, col/row and idx are recorded
last_num = 0

while len(match) == 0: #do this while the match list is empty
    current_numbers = number_stream[0:end_slice] 
    last_num = int(current_numbers[-1])
    # print(current_numbers)
    
    # loop through the grids
    for i in range(len(grids)):
        
        g = grids[i]
        t = [list(j) for j in zip(*grids[i])]
        
        # check the rows and columns
        for k in range(5):
            
            match_row = [val for val in g[k] if val in current_numbers]
            #print(match_row, i, k, 'row', len(match_row))
            
            if len(match_row) > 4:
                #print('match found!')
                match = [i, k, 'row']
                break
                
            match_col = [val for val in t[k] if val in current_numbers]
            #print(match_col, i, k, 'col', len(match_col))
            
            if len(match_col) > 4:
                #print('match found!')
                match = [i, k, 'col']
                break
            
            k += 1
        if len(match) > 0:
            break
        else:
            i += 1
    
    end_slice += 1


print('Winning grid:')
display(grids[match[0]])

ints = [[int(i) for i in j] for j in grids[match[0]]]

'''
After several hours confusion, I realised the winning numbers
are not the only marked numbers on the winning board :lightbulb:
'''

marked_numbers = []
# Find all the marked numbers on the winning grid
for row in grids[match[0]]:
    marked_numbers.append([val for val in row if val in current_numbers])

# Flatten the list of lists and cast to int, get the sum of marked numbers
marked_numbers = [int(val) for sub in marked_numbers for val in sub]
marked_sum = sum(marked_numbers)

# Get the sum of all the number in the winnng grid

grid_total = sum([sum(i) for i in ints])

score = (grid_total - marked_sum) * last_num
print('Winning board score: ', score)


Winning grid:


[['35', '30', '9', '51', '34'],
 ['98', '80', '56', '62', '85'],
 ['93', '36', '18', '65', '12'],
 ['54', '32', '26', '79', '49'],
 ['5', '83', '41', '60', '89']]

Winning board score:  35670


In [ ]:
'''
Part 2 - Let the Squid win!

Goals:
1. Identify the grid that wins last
2. Calculate the winning score of that board

Thoughts:

1. Can't just search for the last winning row or column:
    for the grid to be the last winner, it can't already have a
    completed row/column
2. So... once a grid wins, add it to a list of winners and remove it 
    from the grids in play
3. Since the grid is removed on a win, need to iterate over a copy,
    refresh each time the number stream advances

Process when a grid wins:

1. Append to list of winning grids
2. Record triggering number and end_slice position
3. Pop grid from main list of grids

Process for calculating score:

1. get numbers called from last winning slice
2. Compare that number stream to the last board in the winners list
    to get the marked numbers
'''

In [83]:
with open('input.txt', 'r') as file:
    number_stream = file.readline()
    number_stream = number_stream.strip().split(',')

with open('input.txt', 'r') as file:
    full = file.readlines()
    num_lines = len(full)
    

line_start = 2
lines_to_read = 5
grids = []

while line_start < num_lines:
    with open('input.txt', 'r') as file:
        block = file.readlines()[line_start:line_start + lines_to_read]
        grid = []
        for line in block:
            grid.append(line.strip().split())
        line_start += 6
        grids.append(grid)

end_slice = 5
last_winning_num = 0
last_winning_slice = 0
winning_grids = []

# the main loop needs to iterate through the entire number stream

while end_slice < len(number_stream) + 1:
    current_numbers = number_stream[0:end_slice]
    last_num = int(current_numbers[-1])
    iter_grid = grids.copy()
    for grid in iter_grid:  # (rem  winners this time, iterate copy)
            
            for i in range(5):
                # check rows 
                match_row = [val for val in grid[i] if val in current_numbers]
                
                if len(match_row) > 4:
                    # winner
                    winning_grids.append(grid)
                    last_winning_num = last_num
                    last_winning_slice = end_slice
                    grids.remove(grid)
                    break
                
                # check columns
                # transpose grid
                t = [list(j) for j in zip(*grid)]
                match_col = [val for val in t[i] if val in current_numbers]
                
                if len(match_col) > 4:
                    #winner
                    winning_grids.append(grid)
                    last_winning_num = last_num
                    last_winning_slice = end_slice
                    grids.remove(grid)
                    break
                
                i += 1
    
    end_slice += 1
            

last_winning_stream = number_stream[0:last_winning_slice]
last_winning_grid = winning_grids[-1]

marked_numbers = []

for row in last_winning_grid:
    marked_numbers.append([val for val in row if val in last_winning_stream])

marked_numbers = [int(val) for sub in marked_numbers for val in sub]
marked_sum = sum(marked_numbers)

ints = [[int(i) for i in j] for j in last_winning_grid]

grid_total = sum([sum(i) for i in ints])

score = (grid_total - marked_sum) * last_winning_num
print('Last winning board score: ', score)

display(last_winning_grid)

Last winning board score:  22704


[['3', '72', '23', '55', '22'],
 ['95', '6', '48', '8', '46'],
 ['13', '5', '35', '57', '45'],
 ['33', '79', '38', '88', '36'],
 ['94', '64', '12', '14', '63']]